<a href="https://colab.research.google.com/github/jeanphilippeadielou/CodeML2022/blob/main/cm22_ch6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Code ML 2022 - Challenge 6

## 1 - Pre-processing

### 1.1 - Annotation files
(pandas DataFrames*)

In [ ]:
#Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/MyDrive/colab_nb_jp/'

[Errno 2] No such file or directory: '/MyDrive/colab_nb_jp/'
/content


In [ ]:
!ls

af.csv	drive  sample_data


In [ ]:
!unzip participants_dataset.zip

unzip:  cannot find or open participants_dataset.zip, participants_dataset.zip.zip or participants_dataset.zip.ZIP.


In [ ]:
import os
import pandas as pd
af_csv = pd.read_csv('/content/drive/MyDrive/colab_nb_jp/new_dataset_participants/participants_dataset.csv')
img_direc = '/MyDrive/colab_nb_jp/new_dataset_participants/'

In [ ]:
#Formatting annotations_files


raw_af = pd.DataFrame(af_csv[["filename", "label"]])
raw_af['filename'] = raw_af['filename'].str.replace('files/','')
raw_ldls = pd.DataFrame(raw_af["label"].unique())

#LABELED DATA (IMAGES)
lbld_af = pd.DataFrame(af_csv[["filename", "label"]])
lbld_af['filename'] = raw_af['filename'].str.replace('files/','')
#Remove unlabeled images
lbld_af.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

tot_lab = pd.DataFrame(raw_af["label"].unique())
#print(tot_lab)

part_lab = pd.DataFrame(lbld_af["label"].unique())
#print(part_lab)
# print(len(raw_af))
dictiii = {}
index = 1
for key in ['Tomato_healthy',
                       'Potato___Early_blight',
         'Tomato__Tomato_YellowLeaf__Curl_Virus',
                          'Tomato_Early_blight',
                          'Tomato__Target_Spot',
                         'Potato___Late_blight',
                              'Tomato_Leaf_Mold',
   'Tomato_Spider_mites_Two_spotted_spider_mite',                    
                    'Tomato_Septoria_leaf_spot',
                  'Tomato__Tomato_mosaic_virus',
               'Pepper__bell___Bacterial_spot',
                        'Tomato_Bacterial_spot',
                          'Tomato_Late_blight',
                       'Pepper__bell___healthy',
                            'Potato___healthy',
                                         'NaN'] :
  dictiii[key] = index
  index +=1

raw_af = pd.DataFrame(af_csv[["filename", "label"]])
raw_af['filename'] = raw_af['filename'].str.replace('files/','')


# raw_af['label'] = raw_af['label'].replace(to_replace=dictiii)
lbld_af['label'] = lbld_af['label'].replace(to_replace=dictiii).astype({'label': 'int64'})
#raw_af.to_csv()
print(lbld_af)

                                               filename  label
0     e2991a66-412d-4841-8dc0-524e38338a82 Leaf 517....      1
1     e9392a5e-2fac-48c5-a23d-df3aa3f61048 Leaf 217.JPG      1
2     87a77f4d-38bd-42dc-bdd3-5f2a1fa95ca7 Leaf 469....      1
3         efe6c986-b85c-40f1-8cb5-345acbb36b71 0579.JPG      1
4         cdf10741-0ed4-4a27-a2e9-8970e4426730 0356.JPG      1
...                                                 ...    ...
8211      ff700844-68ad-4e99-8427-58a39c07f817 1860.JPG     15
8212      23f4d3b1-23f4-43fb-96c2-dae37224ec74 1803.JPG     15
8213      170f1f57-0fd4-421f-9c82-3b1804be63ad 1771.JPG     15
8214      b89db9d3-a413-4781-bc88-5f6c2085ab5f 1909.JPG     15
8215      20ac28d6-5708-4e31-8676-1b1979de094f 1735.JPG     15

[8216 rows x 2 columns]


### 1.2 - Splitting Images
(train, test, val)

In [ ]:
!pip install split-folders[full]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import splitfolders

In [ ]:
splitfolders.fixed("/content/drive/MyDrive/colab_nb_jp/new_dataset_participants/","/content/drive/MyDrive/colab_nb_jp/new_dataset_participants/",seed=40, fixed=(6573,1643))

'''#Alternative split;
import string
from torchvision import datasets as ds
from torch.utils.data import DataLoader, random_split
from ds import TESNamesDataset

data_root = '/MyDrive/colab_nb_jp/new_dataset_participants/'
charset = string.ascii_letters + "-' "
length = 30

dataset = TESNamesDataset(data_root, charset, length)
trainset, valset = random_split(dataset, [15593, 3898])

#train_loader = DataLoader(trainset, batch_size=10, shuffle=True, num_workers=2)
val_loader = DataLoader(valset, batch_size=10, shuffle=True, num_workers=2)'''

ValueError: ignored

In [ ]:
#Verify that we have the right amount of files for train and test
_, _, files = next(os.walk("/content/drive/MyDrive/colab_nb_jp/new_dataset_participants/train"))
file_train_count = len(files)
print(file_train_count)

_, _, files = next(os.walk("/content/drive/MyDrive/colab_nb_jp/new_dataset_participants/test"))
file_test_count = len(files)
print(file_test_count)

StopIteration: ignored

##2 - Dataset Creation

In [ ]:
from torchvision.io import read_image

In [ ]:
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset
from PIL import Image

### 2.1 - Custom Dataset class

In [ ]:
class Custom_dataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = annotations_file
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        #imcol = self.img_labels['filename']
        image = Image.open(img_path)
        #image = T.ToPILImage()(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

### 2.2 - Dataset Instanciation and DataLoad

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch
from torchvision.transforms import Lambda

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#batch_size = 4

training_data = Custom_dataset(
    lbld_af,
    "/content/drive/MyDrive/colab_nb_jp/new_dataset_participants/train",
    transform=transform,
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.int64).scatter_(0, torch.tensor(y), value=1))
    )

trainloader = torch.utils.data.DataLoader(training_data, 
                                          #batch_size=batch_size,
                                          shuffle=False, 
                                          num_workers=0
                                          )

'''testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)'''
                            
test_data = Custom_dataset(
    lbld_af,
    "/content/drive/MyDrive/colab_nb_jp/new_dataset_participants/test",
    transform=transform)

testloader = torch.utils.data.DataLoader(test_data, 
                                         #batch_size=batch_size,
                                         shuffle=False,
                                         num_workers=0
                                         )

print(len(trainloader))

8216


In [ ]:
print(training_data.img_dir)

/content/drive/MyDrive/colab_nb_jp/new_dataset_participants/train


###Labels

In [ ]:
#labels_map = {

labels_map = {
0: 'Pepper bell Bacterial spot', 
1: 'Pepper bell healthy', 
2: 'Potato Early blight', 
3: 'Potato Late_blight', 
4: 'Potato healthy', 
5: 'Tomato Bacterial spot', 
6: 'Tomato Early blight', 
7: 'Tomato Late blight', 
8: 'Tomato Leaf Mold', 
9: 'Tomato Septoria leaf spot', 
10: 'Tomato Spider mites Two spotted spider mite', 
11: 'Tomato TargetSpot', 
12: 'Tomato Tomato YellowLeaf Curl Virus', 
13: 'Tomato Tomato mosaic virus', 
14: 'Tomato healthy',
}
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    #image = Image.open(training_data.img_labels['filename'].iloc[sample_idx,0])
    imcol = training_data.img_labels['filename']
    print(imcol[sample_idx])
    img = Image.open(str(imcol.iloc[sample_idx]))
    label = training_data.img_labels.iloc[sample_idx, 1]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

47082588-ba81-435b-a99b-6563eb2dbe7a 9817.JPG


FileNotFoundError: ignored

<Figure size 576x576 with 0 Axes>

##2 - Model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(3721, 120) #16 *16 en principe...
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()

In [ ]:
model = Net()
print(model)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### 2.-1 - viz

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
#print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

TypeError: ignored

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

#3-Train

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        print(data)
        # get the inputs; data is a list of [inputs, labels]
        inputs = data[0][0]
        labels = data[1][0]
        print(inputs.size(), labels)

      
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

FileNotFoundError: ignored